In [3]:
!pip install contractions
import nltk
from nltk.corpus import movie_reviews
from nltk.corpus import stopwords
import contractions
import numpy as np
from gensim.models import LsiModel
from gensim import models
from gensim import corpora
from gensim.similarities import MatrixSimilarity

nltk.download('movie_reviews')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
#loading the corpus
sents = nltk.Text(movie_reviews.sents())
print(sents[0])
print("No. of Documents: ",len(sents))

['plot', ':', 'two', 'teen', 'couples', 'go', 'to', 'a', 'church', 'party', ',', 'drink', 'and', 'then', 'drive', '.']
No. of Documents:  71532


In [6]:
ps = nltk.PorterStemmer()
lemmatizer = nltk.stem.WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

In [7]:

#remove contraction and stop words
def removeContraction(sents_list):
  new_sents = []
  for s in sents_list:
    words = []
    for word in s:
      word = contractions.fix(word)
      word_split = word.split(' ')
      for w in word_split:
        if not w.lower() in stop_words:
          words.append(w.lower())
    if len(words) != 0:
      new_sents.append(words)

  return new_sents

In [8]:
sents = removeContraction(sents) #removing contraction for eg. I'll -> I will 

#pre-processing
modified_sents = []
for sent in sents:
  words = []
  for word in sent:
       word = ps.stem(word) #performing stemming for e.g several -> sever
       word = lemmatizer.lemmatize(word, pos="v") #performing lemmitization for eg. running -> run
       if word.isalpha():
        words.append(word)
  if len(words) != 0:
    modified_sents.append(words)

print(modified_sents[0])
print(len(modified_sents))

['plot', 'two', 'teen', 'coupl', 'go', 'church', 'parti', 'drink', 'drive']
67018


In [9]:
#getting the vocabulary
vocab = corpora.Dictionary(modified_sents)
print(vocab)

Dictionary(25037 unique tokens: ['church', 'coupl', 'drink', 'drive', 'go']...)


In [10]:
doc_term_matrix = [vocab.doc2bow(tokens) for tokens in modified_sents]
tfidf = models.TfidfModel(doc_term_matrix)
corpus_tfidf = tfidf[doc_term_matrix]

In [12]:
nltk.corpus.movie_reviews.fileids()
lsi = models.LsiModel(corpus_tfidf, id2word=vocab)  # initialize an LSI transformation


In [14]:
from pprint import pprint
pprint(lsi.print_topics(num_topics=3, num_words=25))

[(0,
  '0.315*"film" + 0.308*"movi" + 0.218*"one" + 0.170*"like" + 0.169*"see" + '
  '0.169*"make" + 0.147*"good" + 0.141*"get" + 0.140*"charact" + 0.123*"time" '
  '+ 0.116*"go" + 0.105*"even" + 0.103*"know" + 0.103*"much" + 0.100*"well" + '
  '0.098*"would" + 0.096*"realli" + 0.095*"scene" + 0.094*"think" + '
  '0.091*"stori" + 0.089*"end" + 0.088*"thing" + 0.088*"say" + 0.085*"bad" + '
  '0.081*"way"'),
 (1,
  '-0.849*"movi" + 0.427*"film" + 0.100*"one" + -0.093*"see" + 0.089*"charact" '
  '+ -0.074*"good" + -0.068*"bad" + 0.052*"stori" + 0.049*"scene" + '
  '-0.040*"like" + 0.035*"get" + 0.034*"much" + 0.033*"even" + 0.032*"perform" '
  '+ 0.030*"take" + 0.027*"two" + 0.027*"littl" + 0.027*"give" + 0.026*"best" '
  '+ 0.026*"mr" + 0.025*"interest" + 0.025*"also" + -0.025*"funni" + '
  '0.024*"way" + 0.024*"actor"'),
 (2,
  '0.718*"see" + -0.530*"good" + 0.203*"go" + 0.128*"get" + -0.123*"make" + '
  '0.120*"know" + -0.110*"bad" + 0.102*"one" + -0.085*"charact" + 0.081*"want" '
  '+

topic 0 : good = 0.147 bad= 0.085 (good reviews)

topic 1: good= -0.074 bad= -0.068 (mixed reviews)

topic 2: good= -0.53 bad= -0.11 (bad reviews)